In [1]:
from nltk.translate.bleu_score import SmoothingFunction
from nltk.translate import bleu
import numpy as np
from collections import Counter
import nltk
from nltk.lm.preprocessing import pad_both_ends, padded_everygram_pipeline
from nltk.lm.models import Laplace
from nltk.lm import Vocabulary

from utils.evaluation import *

c:\users\user\anaconda3\envs\py36\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
c:\users\user\anaconda3\envs\py36\lib\site-packages\numpy\.libs\libopenblas.QVLO2T66WEPI7JZ63PS3HMOHFEY472BC.gfortran-win_amd64.dll
c:\users\user\anaconda3\envs\py36\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
with open("couplet/test/out.txt", encoding = 'utf8') as f:
    gold = [line.strip().split() for line in f.readlines()]
# with open("couplet/output.txt", encoding = 'utf8') as f:
#     y_pred = [line.strip().split() for line in f.readlines()]
with open("couplet/train/in.txt", encoding = 'utf8') as f:
    tr_in =  [line.strip().split() for line in f.readlines()]
with open("couplet/train/out.txt", encoding = 'utf8') as f:
    tr_out =  [line.strip().split() for line in f.readlines()]
with open("couplet/test/in.txt", encoding = 'utf8') as f:
    te_in = [line.strip().split() for line in f.readlines()]

## Evaluation Usage

In [5]:
from utils.evaluation import *

a = ['夜幕',
 '莺声溅']

evaluate_pred(a, a, 'perplexity_model.pt')

(0.774001598220131, [146.49264761345754, 284.2561369557696])

## Bleu score

In [20]:
smoothfct = SmoothingFunction().method2
bleu_score = 0
for i in range(len(gold)):
    bleu_score += bleu([te_in[i]], gold[i], smoothing_function = smoothfct)
avg_bleu = bleu_score / len(gold)
print("Average bleu score on the test set is: ", avg_bleu)

Average bleu score on the test set is:  0.03177547785577113


## Perplexity

In [8]:
# bigram
n = 2
train_sentences = tr_in + tr_out
words = [word for sentence in train_sentences for word in sentence]
words.extend(["<s>", "<e>"])
vocab = Vocabulary(words)
train_data = [nltk.bigrams(pad_both_ends(t, n = n)) for t in train_sentences]


lm = Laplace(n)
lm.fit(train_data, vocab)

In [35]:
import pickle
f = open('perplexity_model.pt', 'wb')
pickle.dump(lm, f)
f.close()

In [36]:
f = open('perplexity_model.pt', 'rb')
lm = pickle.load(f)
f.close()

In [41]:
test_data = [nltk.bigrams(pad_both_ends(t, n = n)) for t in y_pred]

for i, test in enumerate(test_data[:3]):
    print("PP({0}):{1}".format(''.join(y_pred[i]), lm.perplexity(test)))


PP(和谐南供，安全送电保畅通，争做领头羊):375.9619036414558
PP(夜幕已沉梦更闲):605.4699869560575
PP(莺声溅落柳枝头):150.26441075846543


In [40]:
lm.perplexity(test_data[0])

ZeroDivisionError: division by zero

In [ ]:
#trigram
n = 3
train_sentences = tr_in + tr_out
words = [word for sentence in train_sentences for word in sentence]
words.extend(["<s>", "<e>"])
vocab = Vocabulary(words)
train_data = [nltk.bigrams(pad_both_ends(t, n = n)) for t in train_sentences]


lm = Laplace(n)
lm.fit(train_data, vocab)

test_data = [nltk.bigrams(pad_both_ends(t, n = n)) for t in y_pred]

for i, test in enumerate(test_data):
    print("PP({0}):{1}".format(''.join(y_pred[i]), lm.perplexity(test)))
